<a href="https://colab.research.google.com/github/anwar1406/IGS-Data-Download/blob/main/data_downloader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This module provides utilities to  download IGS station data files
(e.g., observation and navigation files) from the specified FTP server

In [5]:
#import modules
import os
import shutil
import urllib.request
from datetime import datetime, timedelta
from contextlib import closing
from urllib.request import URLError

In [6]:
__all__ = ["gps_week", "igs_station_data"]

def gps_week(day_of_year, year):
    """
    Convert day of year and year to GPS week.

    Parameters
    ----------
    day_of_year : int
        Day of the year (1-365 or 1-366 for leap years).
    year : int
        The year (e.g., 2023).

    Returns
    -------
    int
        The corresponding GPS week number.
    """
    gps_epoch = datetime(1980, 1, 6)

    # Convert year and day of year to a date
    target_date = datetime(year, 1, 1) + timedelta(days=day_of_year - 1)

    # Calculate difference in days from GPS epoch
    days_since_epoch = (target_date - gps_epoch).days

    # Calculate GPS week
    return days_since_epoch // 7

def igs_station_data(station_name, doy_i, doy_f, year, country_name="IND", file_path=""):
    """
    Download IGS station data (observation and navigation files) for a given station and date range.

    Parameters
    ----------
    station_name : str
        Name of the GNSS station (e.g., "IISC").
    doy_i : int
        Start day of the year (DOY).
    doy_f : int
        End day of the year (DOY).
    year : int
        The year of the data (e.g., 2023).
    country_name : str, optional
        Country code (e.g., "IND" for India). Default is "IND".
    file_path : str, optional
        Directory to save the downloaded files. Default is the current directory.

    Returns
    -------
    None
        Downloads the files to the specified directory.
    """
    base_url = r"ftp://gssc.esa.int/gnss/data/daily/"

    for i in range(doy_i, doy_f + 1):  # Include `doy_f`
        day_of_year = f"{i:03}"  # Zero-padded day of year
        path = f"{base_url}{year}/{day_of_year}/"

        # File names
        file_MO = f"{station_name}00{country_name}_R_{year}{day_of_year}0000_01D_30S_MO.crx.sum.gz"
        file_MN = f"{station_name}00{country_name}_R_{year}{day_of_year}0000_01D_MN.rnx.gz"

        # Download observation file
        try:
            with closing(urllib.request.urlopen(path + file_MO)) as r:
                save_path = os.path.join(file_path, file_MO) if file_path else file_MO
                with open(save_path, 'wb') as f:
                    shutil.copyfileobj(r, f)
                    print(f"Observation file for DOY {i} has been downloaded")
        except URLError:
            print(f"Observation file for DOY {i} is missing")

        # Download navigation file
        try:
            with closing(urllib.request.urlopen(path + file_MN)) as r:
                save_path = os.path.join(file_path, file_MN) if file_path else file_MN
                with open(save_path, 'wb') as f:
                    shutil.copyfileobj(r, f)
                    print(f"Navigation file for DOY {i} has been downloaded")
        except URLError:
            print(f"Navigation file for DOY {i} is missing")



**Example usage of igs_station_data function**

In [ ]:
# Example usage of igs_station_data function
if __name__ == "__main__":


    # Parameters
    station_name = "IISC"        # Station name (e.g., "IISC")
    doy_i = 152                  # Start day of the year (DOY)
    doy_f = 156                  # End day of the year (DOY)
    year = 2023                  # Year of the data
    country_name = "IND"         # Country code (e.g., "IND" for India)
    file_path = r"D:Lab2\Data"  # Directory to save the files (ensure it exists)

    # Create the directory if it does not exist
    os.makedirs(file_path, exist_ok=True)

    # Call the function
    igs_station_data(
        station_name=station_name,
        doy_i=doy_i,
        doy_f=doy_f,
        year=year,
        country_name=country_name,
        file_path=file_path
    )